In [1]:
se_cells_meta_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/se_cells_meta.tsv"
sample_names = "preB,postB"
outdir= "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/dominant_clone_clust/"
cluster_names_f = "/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/DUPI_april08_2021/v2_MTblacklist_mtasnucl_Bonly/results/atac_clusters.txt" #"None"

In [2]:
samples = sample_names.split(",")
samples

['preB', 'postB']

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from os.path import join, basename, exists
import click
from glob import glob
from mplh.fig_utils import helper_save as hs
from mplh.cluster_help import plot_cluster as pc
import logging
import os
from matplotlib import rcParams
#import scanpy
#rcParams['lines.markersize'] = 10
import numpy as np
from sklearn.manifold import TSNE

from scipy import stats

In [4]:
np.random.seed(42)

In [5]:
if not exists(outdir):
    os.mkdir(outdir)

In [6]:
cells_meta = pd.read_csv(se_cells_meta_f,sep="\t")
cells_meta = cells_meta[["condition", "lineage", "name", "donor", "seurat_clusters"]]

if not (cluster_names_f is None or cluster_names_f == "None"):
    print("Renaming clusters")
    cluster_names = pd.read_csv(cluster_names_f, sep=",", header=None, dtype=str).set_index(0)
    cells_meta["cluster_names"] = cells_meta["seurat_clusters"].astype(str).map(cluster_names[1])
    cells_meta["seurat_clusters"] = cells_meta["cluster_names"]
cells_meta

Renaming clusters


,condition,lineage,name,donor,seurat_clusters,cluster_names
preB_AAACGAAGTCACTCTC-1,preB,10,0_10,0,0,0
preB_AAACGAAGTCCCTAAA-1,preB,33,3_33,3,NK,NK
preB_AAACGAAGTCTGTGTA-1,preB,20,1_20,1,2,2
preB_AAACTCGCAACTCGTA-1,preB,0,2_0,2,NK,NK
preB_AAACTCGCAGAACGAC-1,preB,9,2_9,2,DC;monocyte,DC;monocyte
...,...,...,...,...,...,...
postB_TTTGGTTTCGAGGTCC-1,postB,9,3_9,3,CD8+_A,CD8+_A
postB_TTTGGTTTCTAGCTGA-1,postB,21,3_21,3,0,0
postB_TTTGTGTCACAGTTCA-1,postB,4,1_4,1,0,0
postB_TTTGTGTGTGCCGGTT-1,None,None,None,None,CD4,CD4


## Get counts
(and add 1 to all for pseudo)

In [7]:
counts = cells_meta.groupby(["condition","name","donor", "seurat_clusters"]).size().reset_index().rename({0:"count"},axis=1)
counts = counts[~(counts["name"]=="None")]
counts


se_clust = np.unique(counts["seurat_clusters"].values)
clones = np.unique(counts["name"].values)

for name in clones:
    donor = name.split("_")[0]
    for clust in se_clust:
        for sample in samples:
            curr = counts.loc[(counts["name"]==name) & (counts["condition"]==sample) 
                          & (counts["seurat_clusters"]==clust) ]
            if len(curr) == 0:
                # Add to counts
                counts.append(pd.Series({"condition":sample, "name":name, "seurat_clusters":clust,"donor": donor,
                          "count": 0, "ID": f"{name}_{sample}"}), ignore_index=True)
counts        

counts["count"]+=1

In [8]:
# Setup the per-donor per-condition batch for normalization
counts["batch"] = np.log2(counts.groupby(["donor", "condition"]).transform(np.sum)["count"].astype(int))
counts["log"] = np.log2(counts["count"])

# Setup the normalized counts 
counts["norm"] = counts["log"]/counts["batch"]
counts["ID"] = counts["name"] + "_" + counts["condition"]
meta = counts[~(counts.duplicated(subset="ID"))][["ID", "condition", "donor", "name"]].set_index("ID")

clust_clones_counts_log = counts.pivot(columns="seurat_clusters",values="log", index="ID").fillna(0)
clust_clones_counts = counts.pivot(columns="seurat_clusters",values="count", index="ID").fillna(0)

clust_clones_counts_norm = counts.pivot(columns="seurat_clusters",values="norm", index="ID").fillna(0)
clust_clones_counts_norm

seurat_clusters,0,1,2,3,4,5,6,7,8,9,10
ID,,,,,,,,,,,
0_0_postB,0.284123,0.109914,0.308567,0.109914,0.109914,0.109914,0.174209,0.174209,0.000000,0.000000,0.000000
0_0_preB,0.227586,0.294150,0.113793,0.227586,0.227586,0.227586,0.113793,0.227586,0.000000,0.000000,0.000000
0_10_postB,0.219828,0.174209,0.109914,0.109914,0.000000,0.000000,0.000000,0.109914,0.000000,0.109914,0.000000
0_10_preB,0.227586,0.180357,0.000000,0.227586,0.113793,0.227586,0.113793,0.113793,0.000000,0.000000,0.000000
0_11_postB,0.000000,0.329742,0.000000,0.000000,0.219828,0.000000,0.000000,0.255212,0.219828,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
3_5_postB,0.359019,0.359019,0.298159,0.400255,0.179510,0.251974,0.208404,0.089755,0.208404,0.089755,0.000000
3_6_preB,0.418322,0.455888,0.418322,0.327585,0.369892,0.233376,0.184946,0.418322,0.233376,0.301634,0.000000
3_7_postB,0.332132,0.350662,0.284516,0.321768,0.251974,0.341728,0.089755,0.142258,0.089755,0.142258,0.000000


In [9]:
clust_clones_counts_norm.to_csv(join(outdir, "cluster_clone_counts_normalized.csv"))
clust_clones_counts.to_csv(join(outdir, "cluster_clone_counts_pseudo1.csv"))
clust_clones_counts_log.to_csv(join(outdir, "cluster_clone_counts_log.csv"))

In [10]:
meta.to_csv(join(outdir, "cluster_clone_meta.csv"))

In [11]:
max_sizes = clust_clones_counts_log.max(axis=1)
dom_clone = clust_clones_counts_log.apply(lambda x: x[(x==max_sizes.loc[x.name])].index.tolist(), axis=1)
dom_clone

ID
0_0_postB           [2]
0_0_preB            [1]
0_10_postB          [0]
0_10_preB     [0, 3, 5]
0_11_postB          [1]
                ...    
3_5_postB           [3]
3_6_preB            [1]
3_7_postB           [1]
3_8_postB        [1, 3]
3_9_postB           [0]
Length: 299, dtype: object

In [12]:
def fill_dom_df(clone):
    dom_clust_names = ";".join([str(cl) for cl in clone["dominant_cluster"]])
    dom_clust = clone["dominant_cluster"][0]
    clone = clone.drop("dominant_cluster").astype(float) # So can do the counts
    cluster_size = clone.loc[dom_clust]
    total_size = clone.sum()
    entropy = stats.entropy(clone/(clone.sum()))
    
    return pd.Series({"dominant_cluster":dom_clust_names, "dominant_cluster_size":cluster_size, 
                      "total_size":total_size, "entropy":entropy}, name=clone.name)


In [13]:
# dom_prep_df = pd.merge(pd.DataFrame(dom_clone).rename({0:"dominant_cluster"}, axis=1), pd.merge(clust_clones_counts_log, clust_clones_counts_norm, suffixes=("_log", "_normalized"), left_index=True,
#         right_index=True), left_index=True, right_index=True)

dom_prep_df = pd.merge(pd.DataFrame(dom_clone).rename({0:"dominant_cluster"}, axis=1), clust_clones_counts, 
                       left_index=True, right_index=True)

dom_prep_df

,dominant_cluster,0,1,2,3,4,5,6,7,8,9,10
ID,,,,,,,,,,,,
0_0_postB,[2],6.0,2.0,7.0,2.0,2.0,2.0,3.0,3.0,0.0,0.0,0.0
0_0_preB,[1],4.0,6.0,2.0,4.0,4.0,4.0,2.0,4.0,0.0,0.0,0.0
0_10_postB,[0],4.0,3.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
0_10_preB,"[0, 3, 5]",4.0,3.0,0.0,4.0,2.0,4.0,2.0,2.0,0.0,0.0,0.0
0_11_postB,[1],0.0,8.0,0.0,0.0,4.0,0.0,0.0,5.0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3_5_postB,[3],16.0,16.0,10.0,22.0,4.0,7.0,5.0,2.0,5.0,2.0,0.0
3_6_preB,[1],12.0,15.0,12.0,7.0,9.0,4.0,3.0,12.0,4.0,6.0,0.0
3_7_postB,[1],13.0,15.0,9.0,12.0,7.0,14.0,2.0,3.0,2.0,3.0,0.0


In [14]:
# dom_df = pd.DataFrame(columns = ["dominant_cluster", "dominant_cluster_size", "total_size", "entropy"], 
#                      index=dom_clone.index)
dom_df = dom_prep_df.apply(fill_dom_df, axis=1).sort_values(["entropy", "dominant_cluster_size"], ascending=False)

In [15]:
dom_df["donor"] = [x.split("_")[0] for x in dom_df.index]
dom_df["clone"] = [x.split("_")[1] for x in dom_df.index]

In [16]:
dom_df.to_csv(join(outdir, "dominant_cluster.csv"))

In [17]:
dom_df

,dominant_cluster,dominant_cluster_size,total_size,entropy,donor,clone
ID,,,,,,
3_42_postB,1,4.0,25.0,2.268858,3,42
3_14_postB,1,12.0,68.0,2.218702,3,14
3_19_postB,0,12.0,58.0,2.202818,3,19
3_6_preB,1,15.0,84.0,2.185453,3,6
0_7_postB,0;1;4;7;8,3.0,23.0,2.177913,0,7
...,...,...,...,...,...,...
2_39_preB,4,2.0,2.0,0.000000,2,39
2_5_postB,2,2.0,2.0,0.000000,2,5
2_9_postB,4,2.0,2.0,0.000000,2,9


## Make a table of each clone and the dominant cluster for each clone.
## Also include the size, the total size, and the entropy of the distribution for each. 
## Plot countplot, and plot the entropy

## Both conditions together

In [18]:
counts_bothconds = cells_meta.groupby(["name","donor", "seurat_clusters"]).size().reset_index().rename({0:"count"},axis=1)
counts_bothconds = counts_bothconds[~(counts_bothconds["name"]=="None")]
counts_bothconds


se_clust = np.unique(counts_bothconds["seurat_clusters"].values)
clones = np.unique(counts_bothconds["name"].values)

for name in clones:
    donor = name.split("_")[0]
    for clust in se_clust:
            curr = counts_bothconds.loc[(counts_bothconds["name"]==name) & (counts_bothconds["seurat_clusters"]==clust) ]
            if len(curr) == 0:
                # Add to counts_bothconds
                counts_bothconds.append(pd.Series({"name":name, "seurat_clusters":clust,"donor": donor,
                          "count": 0, "ID": f"{name}"}), ignore_index=True)
counts_bothconds        

counts_bothconds["count"]+=1

# Setup the per-donor per-condition batch for normalization
counts_bothconds["batch"] = np.log2(counts_bothconds.groupby(["donor"]).transform(np.sum)["count"].astype(int))
counts_bothconds["log"] = np.log2(counts_bothconds["count"])

# Setup the normalized counts_bothconds 
counts_bothconds["norm"] = counts_bothconds["log"]/counts_bothconds["batch"]
counts_bothconds["ID"] = counts_bothconds["name"]
meta = counts_bothconds[~(counts_bothconds.duplicated(subset="ID"))][["ID", "donor", "name"]].set_index("ID")

clust_clones_counts_bothconds_log = counts_bothconds.pivot(columns="seurat_clusters",values="log", index="ID").fillna(0)
clust_clones_counts_bothconds = counts_bothconds.pivot(columns="seurat_clusters",values="count", index="ID").fillna(0)

clust_clones_counts_bothconds_norm = counts_bothconds.pivot(columns="seurat_clusters",values="norm", index="ID").fillna(0)
clust_clones_counts_bothconds_norm

seurat_clusters,0,1,2,3,4,5,6,7,8,9,10
ID,,,,,,,,,,,
0_0,0.323432,0.286438,0.306094,0.236910,0.236910,0.236910,0.204063,0.263747,0.000000,0.000000,0.000000
0_1,0.263747,0.377561,0.236910,0.306094,0.161716,0.161716,0.161716,0.161716,0.000000,0.102031,0.000000
0_10,0.286438,0.236910,0.102031,0.236910,0.102031,0.204063,0.102031,0.161716,0.000000,0.102031,0.000000
0_11,0.000000,0.306094,0.000000,0.000000,0.236910,0.000000,0.000000,0.236910,0.204063,0.000000,0.000000
0_12,0.204063,0.236910,0.161716,0.000000,0.102031,0.000000,0.161716,0.263747,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
3_58,0.000000,0.087992,0.000000,0.000000,0.000000,0.000000,0.087992,0.000000,0.087992,0.000000,0.000000
3_6,0.315448,0.343775,0.315448,0.247025,0.278928,0.175984,0.139464,0.315448,0.175984,0.227456,0.000000
3_7,0.325609,0.343775,0.278928,0.315448,0.247025,0.335016,0.087992,0.139464,0.087992,0.139464,0.000000


In [19]:
clust_clones_counts_bothconds_norm.to_csv(join(outdir, "combinedConditions_cluster_clone_counts_normalized.csv"))
clust_clones_counts_bothconds.to_csv(join(outdir, "combinedConditions_cluster_clone_counts_pseudo1.csv"))
clust_clones_counts_bothconds_log.to_csv(join(outdir, "combinedConditions_cluster_clone_counts_log.csv"))

meta.to_csv(join(outdir, "combinedConditions_cluster_clone_meta.csv"))

In [20]:
max_sizes = clust_clones_counts_bothconds_log.max(axis=1)
dom_clone_bothconds = clust_clones_counts_bothconds_log.apply(lambda x: x[(x==max_sizes.loc[x.name])].index.tolist(), axis=1)
dom_prep_bothconds_df = pd.merge(pd.DataFrame(dom_clone_bothconds).rename({0:"dominant_cluster"}, axis=1), clust_clones_counts_bothconds,
                                 left_index=True, right_index=True)


dom_bothConds_df = dom_prep_bothconds_df.apply(fill_dom_df, axis=1).sort_values(["entropy", "dominant_cluster_size"], ascending=False)
dom_bothConds_df

,dominant_cluster,dominant_cluster_size,total_size,entropy
ID,,,,
0_7,7,7.0,36.0,2.297906
3_42,1,4.0,25.0,2.268858
3_14,1,12.0,68.0,2.218702
3_19,0,12.0,58.0,2.202818
3_6,1,15.0,84.0,2.185453
...,...,...,...,...
2_38,7,3.0,5.0,0.673012
3_56,7,3.0,5.0,0.673012
0_37,7,4.0,6.0,0.636514


In [21]:
dom_bothConds_df["donor"] = [x.split("_")[0] for x in dom_bothConds_df.index]
dom_bothConds_df["clone"] = [x.split("_")[1] for x in dom_bothConds_df.index]


In [22]:
dom_bothConds_df.sort_values("dominant_cluster_size", ascending=False)

,dominant_cluster,dominant_cluster_size,total_size,entropy,donor,clone
ID,,,,,,
3_0,3,31.0,153.0,2.079417,3,0
3_3,0,26.0,97.0,1.995122,3,3
3_1,0,23.0,113.0,2.095692,3,1
3_4,3,22.0,90.0,2.070203,3,4
3_5,3,22.0,89.0,2.041612,3,5
...,...,...,...,...,...,...
3_57,3;4;6,2.0,6.0,1.098612,3,57
3_55,2;7;9,2.0,6.0,1.098612,3,55
2_36,0;2;8,2.0,6.0,1.098612,2,36


In [23]:
dom_bothConds_df.to_csv(join(outdir, "combinedConditions_dominant_cluster.csv"))